In [ ]:
#Installing FuzzyWuzzy
# uses eg: searching through a list of names for possible slight variations of a certain name
!pip install fuzzywuzzy

#Installing FuzzySearch
!pip install fuzzysearch

#Import
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from fuzzysearch import find_near_matches


     |████████████████████████████████| 112 kB 4.4 MB/s 
  Created wheel for fuzzysearch: filename=fuzzysearch-0.7.3-cp37-cp37m-linux_x86_64.whl size=280766 sha256=3c2544ed34177d108c1832034a03e28f52e44f6a58dc42a2f4971fd49448f0e5
  Stored in directory: /root/.cache/pip/wheels/59/11/b5/bae34cc61880d909103c30a1c547df857f5a46adeabfd922b5
Successfully built fuzzysearch


/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
!pip3 install python-Levenshtein

     |████████████████████████████████| 50 kB 2.9 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149860 sha256=ba96b3a5049df54bca7828c30e00588a38abdf503fed10e2756afc6fe370ff60
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# import skin cancer patients dataset
import pandas as pd

skin_cancer_input = pd.read_csv('/content/gdrive/My Drive/Research Colab/output/preprocessed_seqs.csv', delimiter = ',')
actual_mutated = pd.read_csv('/content/gdrive/My Drive/Research Colab/output/mutated_genes.csv', delimiter = ',')

skin_cancer_input.head()

,GENE,ID_SAMPLE,ERRONEOUS_SEQ
0,BRAF,1545608,ATGGCGGCGCTGAGCGGTGGCGGTGGTGGCGGCGCGGAGCCGGGCC...
1,BRAF,1809617,ATGGCGGCGCTGAGCGGTGGCGGTGGTGGCGGCGCGGAGCCGGGCC...
2,BRAF,2380400,ATGGCGGCGCTGAGCGGTGGCGGTGGTGGCGGCGCGGAGCCGGGCC...
3,BRAF,1540436,ATGGCGGCGCTGAGCGGTGGCGGTGGTGGCGGCGCGGAGCCGGGCC...
4,BRAF,1993577,ATGGCGGCGCTGAGCGGTGGCGGTGGTGGCGGCGCGGAGCCGGGCC...


**Dividing the Data into Training and Testing**

In [ ]:
# # split into train/test
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=50)

# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

**Picking a Random Datapoint from the Mutated Sequence Dataset**

In [ ]:
import random

size=len(skin_cancer_input)
random_datapoint_index = random.randint(0, size)
# random_datapoint_index = 380
print(random_datapoint_index)
print(skin_cancer_input.iloc[random_datapoint_index]['ID_SAMPLE'])

9441
2263025


**Getting the Relevant Reference Gene Sequence from the COSMIC API**

In [ ]:
from string import Template
import requests

BASE_URL = 'https://cancer.sanger.ac.uk/cosmic/'
SEQ_URL = Template('sequence?ln=$gene&type=$_type')

def get_gene_seq(gene, _type):
  URL = BASE_URL + SEQ_URL.substitute(gene=gene,_type=_type)
  print(f'GET ---> {URL}')
  request = requests.get(url = URL)
  sequence = request.text
  sequence = sequence.split('\n')
  sequence = ''.join(sequence[1:len(sequence)-1])
  print(sequence)
  print(len(sequence))
  return sequence;

gene_name = skin_cancer_input.iloc[random_datapoint_index]['GENE']
# gene_name = 'BRAF'
print(gene_name)
gene = get_gene_seq(gene_name,'cdna')




MUC16
GET ---> https://cancer.sanger.ac.uk/cosmic/sequence?ln=MUC16&type=cdna
...
43524


In [ ]:
patient_dna = skin_cancer_input.iloc[random_datapoint_index]['ERRONEOUS_SEQ']
print(len(patient_dna))


43524


**Finding the Similarity Threshold for the Reference Gene**

In [ ]:
# Reference

# https://www.kite.com/python/docs/fuzzywuzzy.process.extractBests
# https://pypi.org/project/fuzzysearch/
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1461236/

nucleotides=len(gene)
mutation_percentage = round(175*100/nucleotides,2)
print('Maximum mutation percentage allowed for '+gene_name+' '+ str(mutation_percentage)+' %')
print('Similarity Threshold for '+gene_name+' '+ str(100-mutation_percentage)+' %')

Maximum mutation percentage allowed for MUC16 0.4 %
Similarity Threshold for MUC16 99.6 %


**Calculating the Similarity Rate of the Patient Gene DNA Sequence**

In [ ]:
# def fuzzy_extract(qs, ls, threshold):
#     '''fuzzy matches 'qs' in 'ls' and returns list of tuples of (word,index)'''
    
#     print('inside fuzzy_extract')
    
#     for word,score in process.extractBests(qs, (ls,), score_cutoff=threshold):
#         #print('word: {}'.format(word))
#         print('score: {}'.format(score))
#         l_dist=len(qs)//2
#         print(l_dist)
#         for match in find_near_matches(qs, word, max_l_dist=l_dist):
#             match = word[match.start:match.end]
#             print('match {}'.format(match))
#             index = ls.find(match)
#             yield (match, index)

In [ ]:
print(patient_dna)
print(gene)

ratio = fuzz.ratio(gene, patient_dna)
print('Similarity score: {} %'.format(ratio))

if(ratio>100-mutation_percentage):
  print('Within similarity range')
else:
  print('Sequence has more mutations than the maximum mutations per gene. Therefore fragmented')

...
Similarity score: 0 %
Sequence has more mutations than the maximum mutations per gene. Therefore fragmented



# LSTM-RNN for generating sequence

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

torch.manual_seed(42)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

Neural Network - Defined as a separate class

In [ ]:
class DCNet(nn.Module):

    def __init__(self, hidden_dim, layer1_dim, layer2_dim):
        super(DCNet, self).__init__()
        self.hidden_dim = hidden_dim

        self.lstm = nn.LSTM(4, hidden_dim) 
        self.linear1 = nn.Linear(hidden_dim, layer1_dim)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(layer1_dim, layer2_dim)
        self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(layer2_dim, 4)
        self.hidden_init_values = None
        self.hidden = self.init_hidden()
        nn.init.xavier_uniform(self.linear1.weight)
        nn.init.xavier_uniform(self.linear2.weight)
        nn.init.xavier_uniform(self.linear3.weight)
        
    def init_hidden(self):
        if self.hidden_init_values == None:
            self.hidden_init_values = (autograd.Variable(torch.randn(1, 1, self.hidden_dim)),
                                       autograd.Variable(torch.randn(1, 1, self.hidden_dim)))
        return self.hidden_init_values

    def forward(self, seq):
        lstm_out, self.hidden = self.lstm(
            seq.view(len(seq), 1, -1), self.hidden)
        tmp1 = self.relu1(self.linear1(lstm_out.view(len(seq), -1)))
        tmp2 = self.relu2(self.linear2(tmp1))
        _out = self.linear3(tmp2)
        base_out = _out
        return base_out

In [ ]:
import random
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

# manual one-hot encoding
bmap = {"A":0, "C":1, "G":2, "T":3}
key_list = list(bmap.keys())
val_list = list(bmap.values())

def one_hot(b):
    t = [[0,0,0,0]]
    i = bmap[b]
    t[0][i] = 1
    return t

print("one-hot encoding for DNA bases")
print("A:", one_hot("A"))
print("C:", one_hot("C"))
print("G:", one_hot("G"))
print("T:", one_hot("T"))


def reverse_one_hot(t):
  # print(t)
  max_probability = max(t)
  index = t.index(max_probability)
  # print(index)
  position = val_list.index(index)
  return key_list[position]

print(reverse_one_hot([0,1,0,0]))

one-hot encoding for DNA bases
A: [[1, 0, 0, 0]]
C: [[0, 1, 0, 0]]
G: [[0, 0, 1, 0]]
T: [[0, 0, 0, 1]]
C


Simulate erroneous sequence from an input sequence

In [ ]:
# def sim_error(seq, pi=0.05, pd=0.05, ps=0.01):
#   """
#   Given an input sequence `seq`, generating another
#   sequence with errors. 
#   pi: insertion error rate
#   pd: deletion error rate
#   ps: substitution error rate
#   """
#   out_seq = []
#   for c in seq:
#     while 1:
#       r = random.uniform(0,1)
#     #   # print(r)
#     #   if r < pi:
#     #       out_seq.append(random.choice(["A","C","G","T"]))
#     #   else:
#     #       break
#     # r -= pi
#     # if r < pd:
#     #     continue
#     # r -= pd
#       if r < ps:
#           out_seq.append(random.choice(["A","C","G","T"]))
#           continue
#       r -= ps
#       out_seq.append(c)
#   return "".join(out_seq)

# braf='ATGGCGGCGCTGAGCGGTGGCGGTGGTGGCGGCGCGGAGCCGGGCCAGGCTCTGTTCAACGGGGACATGGAGCCCGAGGCCGGCGCCGGCGCCGGCGCCGCGGCCTCTTCGGCTGCGGACCCTGCCATTCCGGAGGAGGTGTGGAATATCAAACAAATGATTAAGTTGACACAGGAACATATAGAGGCCCTATTGGACAAATTTGGTGGGGAGCATAATCCACCATCAATATATCTGGAGGCCTATGAAGAATACACCAGCAAGCTAGATGCACTCCAACAAAGAGAACAACAGTTATTGGAATCTCTGGGGAACGGAACTGATTTTTCTGTTTCTAGCTCTGCATCAATGGATACCGTTACATCTTCTTCCTCTTCTAGCCTTTCAGTGCTACCTTCATCTCTTTCAGTTTTTCAAAATCCCACAGATGTGGCACGGAGCAACCCCAAGTCACCACAAAAACCTATCGTTAGAGTCTTCCTGCCCAACAAACAGAGGACAGTGGTACCTGCAAGGTGTGGAGTTACAGTCCGAGACAGTCTAAAGAAAGCACTGATGATGAGAGGTCTAATCCCAGAGTGCTGTGCTGTTTACAGAATTCAGGATGGAGAGAAGAAACCAATTGGTTGGGACACTGATATTTCCTGGCTTACTGGAGAAGAATTGCATGTGGAAGTGTTGGAGAATGTTCCACTTACAACACACAACTTTGTACGAAAAACGTTTTTCACCTTAGCATTTTGTGACTTTTGTCGAAAGCTGCTTTTCCAGGGTTTCCGCTGTCAAACATGTGGTTATAAATTTCACCAGCGTTGTAGTACAGAAGTTCCACTGATGTGTGTTAATTATGACCAACTTGATTTGCTGTTTGTCTCCAAGTTCTTTGAACACCACCCAATACCACAGGAAGAGGCGTCCTTAGCAGAGACTGCCCTAACATCTGGATCATCCCCTTCCGCACCCGCCTCGGACTCTATTGGGCCCCAAATTCTCACCAGTCCGTCTCCTTCAAAATCCATTCCAATTCCACAGCCCTTCCGACCAGCAGATGAAGATCATCGAAATCAATTTGGGCAACGAGACCGATCCTCATCAGCTCCCAATGTGCATATAAACACAATAGAACCTGTCAATATTGATGACTTGATTAGAGACCAAGGATTTCGTGGTGATGGAGGATCAACCACAGGTTTGTCTGCTACCCCCCCTGCCTCATTACCTGGCTCACTAACTAACGTGAAAGCCTTACAGAAATCTCCAGGACCTCAGCGAGAAAGGAAGTCATCTTCATCCTCAGAAGACAGGAATCGAATGAAAACACTTGGTAGACGGGACTCGAGTGATGATTGGGAGATTCCTGATGGGCAGATTACAGTGGGACAAAGAATTGGATCTGGATCATTTGGAACAGTCTACAAGGGAAAGTGGCATGGTGATGTGGCAGTGAAAATGTTGAATGTGACAGCACCTACACCTCAGCAGTTACAAGCCTTCAAAAATGAAGTAGGAGTACTCAGGAAAACACGACATGTGAATATCCTACTCTTCATGGGCTATTCCACAAAGCCACAACTGGCTATTGTTACCCAGTGGTGTGAGGGCTCCAGCTTGTATCACCATCTCCATATCATTGAGACCAAATTTGAGATGATCAAACTTATAGATATTGCACGACAGACTGCACAGGGCATGGATTACTTACACGCCAAGTCAATCATCCACAGAGACCTCAAGAGTAATAATATATTTCTTCATGAAGACCTCACAGTAAAAATAGGTGATTTTGGTCTAGCTACAGTGAAATCTCGATGGAGTGGGTCCCATCAGTTTGAACAGTTGTCTGGATCCATTTTGTGGATGGCACCAGAAGTCATCAGAATGCAAGATAAAAATCCATACAGCTTTCAGTCAGATGTATATGCATTTGGAATTGTTCTGTATGAATTGATGACTGGACAGTTACCTTATTCAAACATCAACAACAGGGACCAGATAATTTTTATGGTGGGACGAGGATACCTGTCTCCAGATCTCAGTAAGGTACGGAGTAACTGTCCAAAAGCCATGAAGAGATTAATGGCAGAGTGCCTCAAAAAGAAAAGAGATGAGAGACCACTCTTTCCCCAAATTCTCGCCTCTATTGAGCTGCTGGCCCGCTCATTGCCAAAAATTCACCGCAGTGCATCAGAACCCTCCTTGAATCGGGCTGGTTTCCAAACAGAGGATTTTAGTCTATATGCTTGTGCTTCTCCAAAAACACCCATCCAGGCAGGGGGATATGGTGCGTTTCCTGTCCACTGA'
# print(len(braf))
# print(sim_error(braf,0.05,0.05,0.01))
# print(braf)

In [ ]:
import random

def sim_error(sequence,max_error_percentage):
  length = len(sequence)
  # print(length)
  seq_specific_error = random.uniform(0,max_error_percentage)
  # print(seq_specific_error)
  substitute_count = round(length*seq_specific_error)
  # print(substitute_count)

  s = list(sequence)
  while(substitute_count>0):
    n = random.randint(0,length-1)
    s[n] = random.choice(["A","C","G","T"])
    substitute_count -= 1
  
  return "".join(s)

# error = sim_error('ATGGCGGCGCTGAGCGGTGGCGGTGGTGGCGGCGCGGAGCCGGGCCAGGCTCTGTTCAACGGGGACATGGAGCCCGAGGCCGGCGCCGGCGCCGGCGCCGCGGCCTCTTCGGCTGCGGACCCTGCCATTCCGGAGGAGGTGTGGAATATCAAACAAATGATTAAGTTGACACAGGAACATATAGAGGCCCTATTGGACAAATTTGGTGGGGAGCATAATCCACCATCAATATATCTGGAGGCCTATGAAGAATACACCAGCAAGCTAGATGCACTCCAACAAAGAGAACAACAGTTATTGGAATCTCTGGGGAACGGAACTGATTTTTCTGTTTCTAGCTCTGCATCAATGGATACCGTTACATCTTCTTCCTCTTCTAGCCTTTCAGTGCTACCTTCATCTCTTTCAGTTTTTCAAAATCCCACAGATGTGGCACGGAGCAACCCCAAGTCACCACAAAAACCTATCGTTAGAGTCTTCCTGCCCAACAAACAGAGGACAGTGGTACCTGCAAGGTGTGGAGTTACAGTCCGAGACAGTCTAAAGAAAGCACTGATGATGAGAGGTCTAATCCCAGAGTGCTGTGCTGTTTACAGAATTCAGGATGGAGAGAAGAAACCAATTGGTTGGGACACTGATATTTCCTGGCTTACTGGAGAAGAATTGCATGTGGAAGTGTTGGAGAATGTTCCACTTACAACACACAACTTTGTACGAAAAACGTTTTTCACCTTAGCATTTTGTGACTTTTGTCGAAAGCTGCTTTTCCAGGGTTTCCGCTGTCAAACATGTGGTTATAAATTTCACCAGCGTTGTAGTACAGAAGTTCCACTGATGTGTGTTAATTATGACCAACTTGATTTGCTGTTTGTCTCCAAGTTCTTTGAACACCACCCAATACCACAGGAAGAGGCGTCCTTAGCAGAGACTGCCCTAACATCTGGATCATCCCCTTCCGCACCCGCCTCGGACTCTATTGGGCCCCAAATTCTCACCAGTCCGTCTCCTTCAAAATCCATTCCAATTCCACAGCCCTTCCGACCAGCAGATGAAGATCATCGAAATCAATTTGGGCAACGAGACCGATCCTCATCAGCTCCCAATGTGCATATAAACACAATAGAACCTGTCAATATTGATGACTTGATTAGAGACCAAGGATTTCGTGGTGATGGAGGATCAACCACAGGTTTGTCTGCTACCCCCCCTGCCTCATTACCTGGCTCACTAACTAACGTGAAAGCCTTACAGAAATCTCCAGGACCTCAGCGAGAAAGGAAGTCATCTTCATCCTCAGAAGACAGGAATCGAATGAAAACACTTGGTAGACGGGACTCGAGTGATGATTGGGAGATTCCTGATGGGCAGATTACAGTGGGACAAAGAATTGGATCTGGATCATTTGGAACAGTCTACAAGGGAAAGTGGCATGGTGATGTGGCAGTGAAAATGTTGAATGTGACAGCACCTACACCTCAGCAGTTACAAGCCTTCAAAAATGAAGTAGGAGTACTCAGGAAAACACGACATGTGAATATCCTACTCTTCATGGGCTATTCCACAAAGCCACAACTGGCTATTGTTACCCAGTGGTGTGAGGGCTCCAGCTTGTATCACCATCTCCATATCATTGAGACCAAATTTGAGATGATCAAACTTATAGATATTGCACGACAGACTGCACAGGGCATGGATTACTTACACGCCAAGTCAATCATCCACAGAGACCTCAAGAGTAATAATATATTTCTTCATGAAGACCTCACAGTAAAAATAGGTGATTTTGGTCTAGCTACAGTGAAATCTCGATGGAGTGGGTCCCATCAGTTTGAACAGTTGTCTGGATCCATTTTGTGGATGGCACCAGAAGTCATCAGAATGCAAGATAAAAATCCATACAGCTTTCAGTCAGATGTATATGCATTTGGAATTGTTCTGTATGAATTGATGACTGGACAGTTACCTTATTCAAACATCAACAACAGGGACCAGATAATTTTTATGGTGGGACGAGGATACCTGTCTCCAGATCTCAGTAAGGTACGGAGTAACTGTCCAAAAGCCATGAAGAGATTAATGGCAGAGTGCCTCAAAAAGAAAAGAGATGAGAGACCACTCTTTCCCCAAATTCTCGCCTCTATTGAGCTGCTGGCCCGCTCATTGCCAAAAATTCACCGCAGTGCATCAGAACCCTCCTTGAATCGGGCTGGTTTCCAAACAGAGGATTTTAGTCTATATGCTTGTGCTTCTCCAAAAACACCCATCCAGGCAGGGGGATATGGTGCGTTTCCTGTCCACTGA',0.05)
# len(error)

In [ ]:
# seq = patient_dna
seq = actual_mutated.iloc[random_datapoint_index]['MUTATED_SEQ']

In [ ]:
seq_t = Variable(torch.FloatTensor([one_hot(c) for c in seq]))
print(seq_t)

tensor([[[1., 0., 0., 0.]],

        [[0., 0., 0., 1.]],

        [[0., 0., 1., 0.]],

        ...,

        [[0., 0., 0., 1.]],

        [[0., 0., 1., 0.]],

        [[1., 0., 0., 0.]]])


Simulate 20 erroneous sequences with insertion error at 5%, deletion errors at 5% and subsitution error at 1%

In [ ]:
# seqs = [sim_error(seq, pi=0.05, pd=0.05, ps=0.01) for _ in range(20)]
seqs = [sim_error(seq, 0.04) for _ in range(19)]
print(seq)
seqs.insert(0,patient_dna,)
print ("\n".join(seqs))
seqs_t = [Variable(torch.FloatTensor([one_hot(c) for c in s]))  for s in seqs]
# print(seqs_t)

...


Initialize the DCNet.

In [ ]:
dcnet = DCNet(32, 12, 12)
dcnet.zero_grad()
dcnet.hidden = dcnet.init_hidden()

# initial the paramerters in the DCNet
for name, param in dcnet.named_parameters():
    if 'bias' in name:
        nn.init.constant(param, 0.0)
    elif 'weight' in name:
        nn.init.xavier_normal(param)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  


In [ ]:
#loss_function = nn.L1Loss()
loss_function = nn.MSELoss()
lr = 0.1
optimizer = optim.SGD(dcnet.parameters(), lr=lr)

Training the Network

In [ ]:
#check parameters----------------------------------------------------------------------
range_ = (1, 200)
# mini_batch_size = 5
mini_batch_size = 10
# mini_batch_size = round(len(seq)/4)
# for epoch in range(7501):
for epoch in range(round(len(seq)/2)):
    for i in range(int(len(seqs_t)/mini_batch_size)):
        loss = 0
        s, e = range_
        for seq2 in random.sample(seqs_t, mini_batch_size):
            dcnet.hidden = dcnet.init_hidden()
            dcnet.zero_grad()
            seq2 = seq2[s-1:e]
            seq_ = seq2.view(-1,4)
            out = dcnet(seq_)
            loss += loss_function(out[:-1], seq_[1:])
        loss.backward()
        optimizer.step()
    if epoch % 250==0:
        print("epoch:", epoch, "loss:", loss.cpu().data/mini_batch_size, "learning rate:", lr)
        lr *= 0.95
        optimizer = optim.SGD(dcnet.parameters(), lr=lr)

epoch: 0 loss: tensor(0.1970) learning rate: 0.1
epoch: 250 loss: tensor(0.1737) learning rate: 0.095
epoch: 500 loss: tensor(0.1639) learning rate: 0.09025
epoch: 750 loss: tensor(0.1247) learning rate: 0.0857375
epoch: 1000 loss: tensor(0.0941) learning rate: 0.08145062499999998
epoch: 1250 loss: tensor(0.0539) learning rate: 0.07737809374999999
epoch: 1500 loss: tensor(0.0346) learning rate: 0.07350918906249998
epoch: 1750 loss: tensor(0.0306) learning rate: 0.06983372960937498
epoch: 2000 loss: tensor(0.0183) learning rate: 0.06634204312890622
epoch: 2250 loss: tensor(0.0159) learning rate: 0.0630249409724609
epoch: 2500 loss: tensor(0.0133) learning rate: 0.05987369392383786
epoch: 2750 loss: tensor(0.0112) learning rate: 0.05688000922764597
epoch: 3000 loss: tensor(0.0106) learning rate: 0.05403600876626367
epoch: 3250 loss: tensor(0.0104) learning rate: 0.05133420832795048
epoch: 3500 loss: tensor(0.0098) learning rate: 0.04876749791155295
epoch: 3750 loss: tensor(0.0091) learni

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
dcnet.hidden = dcnet.init_hidden()
xout = dcnet(seq_t[:250])
x1 = xout[:-1].cpu().data.numpy() 
xx1=np.transpose(seq_t[1:250,0,:].data.cpu().numpy())
xx2=np.transpose(x1)

In [ ]:
plt.figure(figsize=(18,3))
plt.subplot(3,1,1)
plt.matshow(xx1, vmin=-0.1, vmax=1.1, fignum=False)
plt.text(0,6,"the tensor for the original template")
frame = plt.gca()
frame.axes.yaxis.set_ticklabels([])

plt.subplot(3,1,2)
plt.matshow(xx2, vmin=-0.1, vmax=1.1, fignum=False)
plt.text(0,6,"reconstructed tensor from the DCNet")
frame = plt.gca()
frame.axes.yaxis.set_ticklabels([])

plt.subplot(3,1,3)
plt.matshow(xx1-xx2, vmin=-0.1, vmax=1.1, fignum=False)
plt.text(0,6,"differences");
frame = plt.gca()
frame.axes.yaxis.set_ticklabels([]);

In [ ]:
dcnet.hidden = dcnet.init_hidden()
base_t= Variable(torch.FloatTensor([one_hot(seq[0])]))

new_ratio = ratio
while(new_ratio<100-mutation_percentage):
  consensus = []

  for _ in range(201):
    xout = dcnet(base_t)
    next_t = [0,0,0,0]
    next_t[np.argmax(xout.cpu().data.numpy())]=1
    consensus.append(next_t)
    base_t= Variable(torch.FloatTensor([next_t]))
 
  derived_output = "".join(reverse_one_hot(c) for c in consensus)
  print(derived_output)
  new_ratio = fuzz.ratio(gene, derived_output)
  print('New Similarity score: {} %'.format(new_ratio))

  consensus = np.array(consensus)
  consensus = consensus.transpose()




In [ ]:
import numpy as np

new_ratio = ratio
patient_seq_t = Variable(torch.FloatTensor([one_hot(c) for c in patient_dna]))

while(new_ratio<100-mutation_percentage):
  dcnet.hidden = dcnet.init_hidden()

  xout = dcnet(patient_seq_t[:250])
  # xout = dcnet(seq_t)
  xout_list = xout.detach().numpy().tolist()

  derived_output = "".join(reverse_one_hot(c) for c in xout_list)
  print(derived_output)
  new_ratio = fuzz.ratio(gene, derived_output)
  print('New Similarity score: {} %'.format(new_ratio))

  # x1 = xout[:-1].cpu().data.numpy() 
  # xx1=np.transpose(patient_seq_t[1:250,0,:].data.cpu().numpy())
  # xx2=np.transpose(x1)

  # patient_seq_t = xout